In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/ResNet
!ls

/content/drive/My Drive/ResNet
data	    main.py  model.py	  run.py
main.ipynb  memo.md  __pycache__  show_cifar10.py


In [0]:
import PIL

PIL.PILLOW_VERSION = PIL.__version__# これをやらないと、torchvisionでエラーを吐く。。。

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from model import *

import os

"""
test with cifar10(10 class).
original paper was tested with imagenet2012 wich is classification over 1000 classes.

ref : https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
"""

class args():
    lr = 0.01# learning rate
    maxepoch = 1000
    batchnum = 8

def main(args):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    best_acc = 0  # best test accuracy
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch

    # 前処理(ネット上のをそのまま利用した)
    transform_train = transforms.Compose([# 学習データの前処理
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([# テストデータの前処理
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=1)# バッチサイズをcpuテスト用に極端に小さくしてある。

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=1)

    # classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    net = ResNet_34(10)
    net = net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)# パラメータはネットのもの

    # train
    for epoch in range(args.maxepoch):
        print("epoch : {}".format(epoch))
        net.train()
        total = 0
        correct = 0

        for batch_idx, (inputs, targets) in enumerate(trainloader):

            # target -> onehot
            class_num = 10
            targets_onehot = torch.eye(class_num, device=device, dtype=torch.long)[targets]

            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            out = net(inputs)
            # import pdb; pdb.set_trace()
            loss = criterion(out, targets)
            # loss = criterion(out, targets_onehot)
            loss.backward()
            optimizer.step()

            # # target -> onehot
            # class_num = 10
            # targets_onehot = torch.eye(class_num, device=device)[targets]

            _, predict = out.max(1)
            predict_onehot = torch.eye(class_num, device=device, dtype=torch.long)[predict]

            # import pdb; pdb.set_trace()
            total += targets.size(0)
            correct += predict.eq(targets).sum().item()

            # acc = correct/total
            # print("epochnum : {}, accuracy : {}".format(epoch,acc))

            if batch_idx%100 == 0:
                print("batch_idx : {}, loss : {}".format(batch_idx, loss))
        

        acc = correct/total
        print("epochnum : {}, accuracy : {}, total : {}, correct : {}".format(epoch, acc, total, correct))

In [0]:
args = args()
main(args)

Streaming output truncated to the last 5000 lines.
batch_idx : 100, loss : 1.7815873622894287
batch_idx : 200, loss : 1.7061430215835571
batch_idx : 300, loss : 1.7825219631195068
batch_idx : 400, loss : 1.6465492248535156
epochnum : 200, accuracy : 0.76692, total : 50000, correct : 38346
epoch : 201
batch_idx : 0, loss : 1.7118617296218872
batch_idx : 100, loss : 1.7282278537750244
batch_idx : 200, loss : 1.6604852676391602
batch_idx : 300, loss : 1.6815975904464722
batch_idx : 400, loss : 1.6713757514953613
epochnum : 201, accuracy : 0.76702, total : 50000, correct : 38351
epoch : 202
batch_idx : 0, loss : 1.6846399307250977
batch_idx : 100, loss : 1.686845064163208
batch_idx : 200, loss : 1.6430401802062988
batch_idx : 300, loss : 1.757470965385437
batch_idx : 400, loss : 1.7483388185501099
epochnum : 202, accuracy : 0.76348, total : 50000, correct : 38174
epoch : 203
batch_idx : 0, loss : 1.7146016359329224
batch_idx : 100, loss : 1.673376441001892
batch_idx : 200, loss : 1.6102492